# ZVP-based reverse-engineering

In [1]:
import sympy
from sympy import FF, sympify, symbols

from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.coordinates import AffineCoordinateModel
from pyecsca.ec.curve import EllipticCurve
from pyecsca.ec.params import DomainParameters
from pyecsca.ec.point import Point
from pyecsca.ec.mod import Mod, SymbolicMod
from pyecsca.ec.context import DefaultContext, local
from pyecsca.misc.cfg import getconfig

In [2]:
cfg = getconfig()
cfg.ec.mod_implementation = "python"

In [3]:
model = ShortWeierstrassModel()
coordsaff = AffineCoordinateModel(model)
coords = model.coordinates["projective"]
add = coords.formulas["add-2007-bl"]
dbl = coords.formulas["dbl-2007-bl"]
neg = coords.formulas["neg"]

# A 64-bit prime order curve for testing things out
p = 0xc50de883f0e7b167
a = Mod(0x4833d7aa73fa6694, p)
b = Mod(0xa6c44a61c5323f6a, p)
gx = Mod(0x5fd1f7d38d4f2333, p)
gy = Mod(0x21f43957d7e20ceb, p)
n = 0xc50de885003b80eb
h = 1

field = FF(p)

infty = Point(coords, X=Mod(0, p), Y=Mod(1, p), Z=Mod(0, p))
g = Point(coords, X=gx, Y=gy, Z=Mod(1, p))

curve = EllipticCurve(model, coords, p, infty, dict(a=a,b=b))
params = DomainParameters(curve, g, n, h)

In [25]:
# add
# TODO: this is wrooooong, needs to have X1 and X2...
P1 = Point(coords, **{str(var): SymbolicMod(symbols(var + "1"), p) for var in coords.variables})
P2 = Point(coords, **{str(var): SymbolicMod(symbols(var + "2"), p) for var in coords.variables})
parameters = {str(var): SymbolicMod(var, p) for var in symbols(model.parameter_names)}
with local(DefaultContext()) as ctx:
    res = add(p, P1, P2, **parameters)[0]

In [20]:
res.X.x

2*Z1*Z2*(Y1*Z2 + Y2*Z1)*(Z1**2*Z2**2*(Y1*Z2 + Y2*Z1)**4 + (X1*Z2 + X2*Z1)**2 - (X1*Z2 + X2*Z1 + Z1*Z2*(Y1*Z2 + Y2*Z1)**2)**2 + 2*(-X1*X2*Z1*Z2 + Z1**2*Z2**2*a + (X1*Z2 + X2*Z1)**2)**2)

In [21]:
res.Y.x

-2*Z1**2*Z2**2*(Y1*Z2 + Y2*Z1)**4 + (-X1*X2*Z1*Z2 + Z1**2*Z2**2*a + (X1*Z2 + X2*Z1)**2)*(-3*Z1**2*Z2**2*(Y1*Z2 + Y2*Z1)**4 - 3*(X1*Z2 + X2*Z1)**2 + 3*(X1*Z2 + X2*Z1 + Z1*Z2*(Y1*Z2 + Y2*Z1)**2)**2 - 4*(-X1*X2*Z1*Z2 + Z1**2*Z2**2*a + (X1*Z2 + X2*Z1)**2)**2)

In [22]:
res.Z.x

4*Z1**3*Z2**3*(Y1*Z2 + Y2*Z1)**3

In [24]:
for op_result in ctx.actions.get_by_index([0])[0].op_results:
    print(repr(op_result), "\t", op_result.value.x)

U1 = X1*Z2 	 X1*Z2
U2 = X2*Z1 	 X2*Z1
S1 = Y1*Z2 	 Y1*Z2
S2 = Y2*Z1 	 Y2*Z1
ZZ = Z1*Z2 	 Z1*Z2
T = U1+U2 	 X1*Z2 + X2*Z1
TT = T 	 (X1*Z2 + X2*Z1)**2
M = S1+S2 	 Y1*Z2 + Y2*Z1
t0 = ZZ 	 Z1**2*Z2**2
t1 = a*t0 	 Z1**2*Z2**2*a
t2 = U1*U2 	 X1*X2*Z1*Z2
t3 = TT-t2 	 -X1*X2*Z1*Z2 + (X1*Z2 + X2*Z1)**2
R = t3+t1 	 -X1*X2*Z1*Z2 + Z1**2*Z2**2*a + (X1*Z2 + X2*Z1)**2
F = ZZ*M 	 Z1*Z2*(Y1*Z2 + Y2*Z1)
L = M*F 	 Z1*Z2*(Y1*Z2 + Y2*Z1)**2
LL = L 	 Z1**2*Z2**2*(Y1*Z2 + Y2*Z1)**4
t4 = T+L 	 X1*Z2 + X2*Z1 + Z1*Z2*(Y1*Z2 + Y2*Z1)**2
t5 = t4 	 (X1*Z2 + X2*Z1 + Z1*Z2*(Y1*Z2 + Y2*Z1)**2)**2
t6 = t5-TT 	 -(X1*Z2 + X2*Z1)**2 + (X1*Z2 + X2*Z1 + Z1*Z2*(Y1*Z2 + Y2*Z1)**2)**2
G = t6-LL 	 -Z1**2*Z2**2*(Y1*Z2 + Y2*Z1)**4 - (X1*Z2 + X2*Z1)**2 + (X1*Z2 + X2*Z1 + Z1*Z2*(Y1*Z2 + Y2*Z1)**2)**2
t7 = R 	 (-X1*X2*Z1*Z2 + Z1**2*Z2**2*a + (X1*Z2 + X2*Z1)**2)**2
t8 = 2*t7 	 2*(-X1*X2*Z1*Z2 + Z1**2*Z2**2*a + (X1*Z2 + X2*Z1)**2)**2
W = t8-G 	 Z1**2*Z2**2*(Y1*Z2 + Y2*Z1)**4 + (X1*Z2 + X2*Z1)**2 - (X1*Z2 + X2*Z1 + Z1*Z2*(Y1*Z2 + Y2*